<a href="https://colab.research.google.com/github/Love1117/Machine_learning-Projects/blob/main/Machine_Learning%20Project/01_Supervised-%20Machine%20Learning/01_Linear_Regression/Car_price_prediction/API_Colab_Running_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi uvicorn pyngrok nest-asyncio

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from fastapi import FastAPI, HTTPException
import uvicorn
from pydantic import BaseModel, Field
import pandas as pd
from typing import Literal
import numpy as np
import joblib
from enum import Enum

model = joblib.load("/content/drive/My Drive/Models/Car_price_model-V1.0.0.joblib")
freq_name_encoder = joblib.load("/content/drive/My Drive/Models/frequency_name.joblib")
car_model_year_encoder = joblib.load("/content/drive/My Drive/Models/car-model_and_year_target-mean.joblib")
Scaler = joblib.load("/content/drive/My Drive/Models/Scaled_car_price.joblib")

CarModelYearEnum = Enum(
      "CarModelYearEnum",
      {k: k for k in car_model_year_encoder.keys()},
      type=str)

FreqNameEnum = Enum(
                  "FreqNameEnum",
                  {k: k for k in freq_name_encoder.keys()},
                  type=str)

TRANSMISSION = {"Manual": 0, "Automatic": 1}
FUEL = {"Petrol": 0, "Diesel": 1, "CNG": 2, "LPG": 3}
OWNER = {"First Owner": 0, "Second Owner": 1, "Third Owner": 2, "Fourth & Above Owner": 3, "Test Drive Car": 4}
SELLER = {"Individual": 0, "Dealer": 1, "Trustmark Dealer": 2}


class Base(BaseModel):
  year: int = Field(..., json_schema_extra={"examples": [2014]})
  km_driven: int = Field(..., json_schema_extra={"examples": [120000]})
  transmission: Literal["Manual", "Automatic"]
  mileage: float = Field(..., json_schema_extra={"examples": [20.5]})
  engine: float = Field(..., json_schema_extra={"examples": [1248.0]})
  max_power: float = Field(..., json_schema_extra={"examples": [75.5]})
  seats: float = Field(..., json_schema_extra={"examples": [5.0]})
  fuel: Literal["Petrol", "Diesel", "CNG", "LPG"]
  owner: Literal["First Owner", "Second Owner", "Third Owner", "Fourth & Above Owner", "Test Drive Car"]
  seller_type: Literal["Individual", "Dealer", "Trustmark Dealer"]
  car_model_year: CarModelYearEnum   # dropdown
  freq_name: FreqNameEnum





app = FastAPI(title="Old model Car price price prediction 2025",
              description= "Production-style ML API for Insurance Claim",
              version="1.0.0")

@app.get("/check_health ✅")
async def check_health():
  return {"Status": "health is good",
          "Model_loaded": True}


@app.get("/Model/info")
async def model_info():
  return {"Used_Model": "LinearRegression",
          "Features": ["name",
                       "year",
                       "selling_price",
                       "km_driven",
                       "fuel",
                       "seller_type",
                       "transmission",
                       "owner",
                       "mileage(km/ltr/kg)",
                       "engine",
                       "max_power",
                       "seats"],
          "Version_type": "version 1.0.0"}


@app.post("/predict")
async def predict(data: Base):

  car_model_year_encoded = car_model_year_encoder[data.car_model_year.value]
  freq_name_encoded = freq_name_encoder[data.freq_name.value]


  input_data = pd.DataFrame([{
      "year": data.year,
      "km_driven": data.km_driven,
      "transmission": TRANSMISSION[data.transmission],
      "mileage": data.mileage,
      "engine": data.engine,
      "max_power": data.max_power,
      "seats": data.seats,
      "fuel": FUEL[data.fuel],
      "owner": OWNER[data.owner],
      "seller_type": SELLER[data.seller_type],
      "car_model_year_encoded": car_model_year_encoded,
      "freq_name": freq_name_encoded}])


  scaled_input = Scaler.transform(input_data)

  prediction = model.predict(scaled_input)[0]

  return {"Prediction": round(float(prediction),2)}

In [4]:
import nest_asyncio
nest_asyncio.apply()

import uvicorn
import asyncio

# Create a Uvicorn configuration
config = uvicorn.Config(app, host="0.0.0.0", port=8001, loop="asyncio") # Changed port to 8001

# Create a Uvicorn server instance
server = uvicorn.Server(config)

# Run the server in a separate asyncio task to avoid blocking the notebook
# This allows other cells (like ngrok) to execute.
asyncio.create_task(server.serve())

<Task pending name='Task-1' coro=<Server.serve() running at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69>>

In [5]:
from pyngrok import ngrok

ngrok.set_auth_token("375mWYaMXwrlkyG1PLQUSKPGCTn_7TEuMBd5te7GFBfgjCQnZ")

public_url = ngrok.connect(8001) # Changed port to 8001
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://untelevised-spangly-lottie.ngrok-free.dev" -> "http://localhost:8001"
